In [17]:
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
#specify driver path
DRIVER_PATH = './chromedriver'
driver = webdriver.Chrome(executable_path = DRIVER_PATH)
driver.get('https://au.indeed.com')
initial_search_button = driver.find_element_by_xpath('//*[@id ="jobsearch"]/button')
initial_search_button.click()

#close_popup = driver.find_element_by_id("popover-close-link") 
#close_popup.click()

advanced_search = driver.find_element_by_xpath("//a[contains(text(),'Advanced Job Search')]")
advanced_search.click()

#search data science 
search_job = driver.find_element_by_xpath('//input[@id="as_and"]')
search_job.send_keys(['data science'])
#set display limit of 30 results per page
display_limit = driver.find_element_by_xpath('//select[@id="limit"]//option[@value="30"]')
display_limit.click()
#sort by date
sort_option = driver.find_element_by_xpath('//select[@id="sort"]//option[@value="date"]')
sort_option.click()
search_button = driver.find_element_by_xpath('//*[@id="fj"]')
search_button.click()

#let the driver wait 3 seconds to locate the element before exiting out
driver.implicitly_wait(3) 

titles=[]
companies=[]
locations=[]
links =[]
reviews=[]
salaries = []
descriptions=[]


for i in range(0,20):
    
    job_card = driver.find_elements_by_xpath('//div[contains(@class,"clickcard")]')
    
    for job in job_card:
       
    #.  not all companies have review
        try:
            review = job.find_element_by_xpath('.//span[@class="ratingsContent"]').text
        except:
            review = "None"
        reviews.append(review)
   #.   not all positions have salary
        try:
            salary = job.find_element_by_xpath('.//span[@class="salaryText"]').text
        except:
            salary = "None"
    #.  tells only to look at the element       
        salaries.append(salary)
        
        try:
            location = job.find_element_by_xpath('.//span[contains(@class,"location")]').text
        except:
            location = "None"
    #.  tells only to look at the element       
        locations.append(location)
        
        try:
            title  = job.find_element_by_xpath('.//h2[@class="title"]//a').text
        except:
            title = job.find_element_by_xpath('.//h2[@class="title"]//a').get_attribute(name="title")
        titles.append(title)
        links.append(job.find_element_by_xpath('.//h2[@class="title"]//a').get_attribute(name="href"))
        companies.append(job.find_element_by_xpath('.//span[@class="company"]').text)
        
    
    try:
        next_page = driver.find_element_by_xpath('//a[@aria-label={}]//span[@class="pn"]'.format(i+2))
        next_page.click()

    except:
        next_page = driver.find_element_by_xpath('//a[@aria-label="Next"]//span[@class="np"]')
        next_page.click()
    #except:
        #next_page = driver.find_element_by_xpath('//a[.//span[contains(text(),"Next")]]')
        #next_page.click()
        
    
    print("Page: {}".format(str(i+2)))

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//a[contains(text(),'Advanced Job Search')]"}
  (Session info: chrome=102.0.5005.115)


In [4]:
########################################################
# STEP 2 - Crawl Indeed ads
########################################################
link_list = []
original_link = 'https://au.indeed.com'
middle = '/jobs?q=%22Big+Data+Analytics%22&start='
for i in range(10):
    if i == 0:
        new = "https://au.indeed.com/jobs?q=%22Big%20Data%20Analytics%22"
    else:
        new = (original_link + middle + str(i*10))
    link_list.append(new)

link_list

['https://au.indeed.com/jobs?q=%22Big%20Data%20Analytics%22',
 'https://au.indeed.com/jobs?q=%22Big+Data+Analytics%22&start=10',
 'https://au.indeed.com/jobs?q=%22Big+Data+Analytics%22&start=20',
 'https://au.indeed.com/jobs?q=%22Big+Data+Analytics%22&start=30',
 'https://au.indeed.com/jobs?q=%22Big+Data+Analytics%22&start=40',
 'https://au.indeed.com/jobs?q=%22Big+Data+Analytics%22&start=50',
 'https://au.indeed.com/jobs?q=%22Big+Data+Analytics%22&start=60',
 'https://au.indeed.com/jobs?q=%22Big+Data+Analytics%22&start=70',
 'https://au.indeed.com/jobs?q=%22Big+Data+Analytics%22&start=80',
 'https://au.indeed.com/jobs?q=%22Big+Data+Analytics%22&start=90']

In [5]:
# Visit each page of results for the search 
for url in link_list:
    print(url)
    uClient_all_jobs = uReq(url)
    page_html_all_jobs = uClient_all_jobs.read()
    page_soup_all = soup(page_html_all_jobs, "html.parser")
    
    a_tag_list = []
    
    
    # Visit each ad on a page of results 
    for data in page_soup_all.findAll('a', {'class':'result'}):
#         print(data)
        link = data.get('id').split("_")[1]

        # Get specific link of job
        new_link = original_link + "/viewjob?jk=" + link

        # Retrieve HTML of this job 
        print(new_link)
        uClient = uReq(new_link)
        specific_course_html = uClient.read()
        page_soup_specific = soup(specific_course_html, "html.parser")
        

        # Get details of this job
        overview = page_soup_specific.findAll('div', {'id':'jobDescriptionText'})[0].text
#         print(overview)
        
        
        # Get title of this job
        title = page_soup_specific.findAll('h1', {'class':'jobsearch-JobInfoHeader-title'})[0].text
        
        
        # Get advertiser's name for this job
        advertiser = page_soup_specific.findAll('div', {'class':'jobsearch-InlineCompanyRating'})[0].text
        
        
        # Append all fields to the dataframe 
        dfObj = dfObj.append({'Name':title, 'Link':new_link, 'Advertiser':advertiser, 'Description':overview}, ignore_index=True)

https://au.indeed.com/jobs?q=%22Big%20Data%20Analytics%22
https://au.indeed.com/jobs?q=%22Big+Data+Analytics%22&start=10
https://au.indeed.com/jobs?q=%22Big+Data+Analytics%22&start=20
https://au.indeed.com/jobs?q=%22Big+Data+Analytics%22&start=30
https://au.indeed.com/jobs?q=%22Big+Data+Analytics%22&start=40
https://au.indeed.com/jobs?q=%22Big+Data+Analytics%22&start=50
https://au.indeed.com/jobs?q=%22Big+Data+Analytics%22&start=60
https://au.indeed.com/jobs?q=%22Big+Data+Analytics%22&start=70
https://au.indeed.com/jobs?q=%22Big+Data+Analytics%22&start=80
https://au.indeed.com/jobs?q=%22Big+Data+Analytics%22&start=90


In [6]:
dfObj

,Name,Link,Advertiser,Description
